# Traffic analysis - Madrid Central

In this notebook we want to analyze if there is a significant change in the use of traffic inside the area of Madrid Central when the measure was instaured. For that we analyze data from 2016 until 2021.

## Imports

In [76]:
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt
from matplotlib.path import Path
import json
import zipfile
import io
import os
import utm
import seaborn as sns

from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import HoverTool, Legend, ColumnDataSource, Title
from bokeh.tile_providers import get_provider, CARTODBPOSITRON
from bokeh.models import GeoJSONDataSource
from bokeh.transform import linear_cmap

output_notebook()
np.random.seed(42)

Loading BokehJS ...

## Data download

The data we want to work with is very large, thus we need to download it from the source as it is not possible to upload it to the version control system we use (GitHub). 

In [7]:
def download_data():
    FIRST_MONTH_ID = 32
    LAST_MONTH_ID = 103
    DATA_PATH = "data"

    LAST_MONTH_ID = 44
    
    for id in range(FIRST_MONTH_ID, LAST_MONTH_ID+1):
        url = f"https://datos.madrid.es/egob/catalogo/208627-{id}-transporte-ptomedida-historico.zip"
        r = requests.get(url)
        z = zipfile.ZipFile(io.BytesIO(r.content))
        z.extractall(DATA_PATH)

In [ ]:
do_download = int(input("WRITE '1' TO DOWNLOAD DATA OR '0' TO NOT "))

if do_download:
    download_data()

## Display location of traffic measurement points

In [12]:
traffic_points = pd.read_csv("shared_data/traffic_points/pmed_trafico_03052016.csv", sep=";")
traffic_points.head()

,idelem,tipo_elem,cod_cent,nombre,st_x,st_y
0,1044,494,03FT08PM01,03FT08PM01,438963.314635,4.474734e+06
1,3600,494,PM30901,PM30901,443729.047369,4.473268e+06
2,3705,494,PM41451,PM41451,439858.261097,4.471574e+06
3,6823,494,PM41453,PM41453,439188.095183,4.470895e+06
4,7033,495,01015,Pº Castellana S-N - Pl. Colon-Hermosilla,441569.555897,4.475502e+06


First we need to calculate the correct *utm* for displaying in `bokeh` maps.

In [13]:
def utm_from_latlon(lat, lon):
    r_major = 6378137.000
    x = r_major * np.radians(lon)
    scale = x/lon
    y = 180.0/np.pi * np.log(np.tan(np.pi/4.0 + 
        lat * (np.pi/180.0)/2.0)) * scale

    return x, y

def get_lat_lon_utm(row):

    # 30 and 'T' is the zone of Madrid
    lat, lon = utm.to_latlon(row["st_x"], row["st_y"], 30, "T")
    
    x, y = utm_from_latlon(lat, lon)

    return pd.Series([lat, lon, x, y])

In [14]:
traffic_points[["latitude", "longitude", "utm_x", "utm_y"]] = traffic_points.apply(get_lat_lon_utm, axis=1)
traffic_points.head()

,idelem,tipo_elem,cod_cent,nombre,st_x,st_y,latitude,longitude,utm_x,utm_y
0,1044,494,03FT08PM01,03FT08PM01,438963.314635,4.474734e+06,40.421001,-3.719488,-414051.481782,4.927311e+06
1,3600,494,PM30901,PM30901,443729.047369,4.473268e+06,40.408129,-3.663184,-407783.811885,4.925429e+06
2,3705,494,PM41451,PM41451,439858.261097,4.471574e+06,40.392598,-3.708640,-412843.963659,4.923159e+06
3,6823,494,PM41453,PM41453,439188.095183,4.470895e+06,40.386431,-3.716471,-413715.710072,4.922257e+06
4,7033,495,01015,Pº Castellana S-N - Pl. Colon-Hermosilla,441569.555897,4.475502e+06,40.428107,-3.688839,-410639.639249,4.928350e+06


Then load the districts information to display them in the map.

In [28]:
with open("shared_data/districts/districts.geojson", "r") as geojson:
    geodata = json.load(geojson)

In [92]:
df_districts = pd.DataFrame([], columns=["name", "latitude",
                                         "longitude", "utm_x",
                                         "utm_y"])
for district in geodata["features"]:
    # Get district name
    district_name = district["properties"]["NOMBRE"]
    # Get district coordinates
    district_coord = district["geometry"]["coordinates"][0]
    df_district = pd.DataFrame(district["geometry"]["coordinates"][0], columns=["st_x", "st_y"])
    df_district["name"] = district_name
    # Calculate correct utm
    df_district[["latitude", "longitude", "utm_x", "utm_y"]] = df_district.apply(get_lat_lon_utm, axis=1)
    df_district = df_district.drop(columns=["st_x", "st_y"])
    # Append to all districts dataframe
    df_districts = pd.concat([df_districts, df_district]).reset_index(drop=True)


district_name = df_districts["name"].unique()
df_districts

,name,latitude,longitude,utm_x,utm_y
0,Centro,40.407345,-3.693162,-411120.867401,4925314.358763
1,Centro,40.407196,-3.693202,-411125.341089,4925292.526182
2,Centro,40.406986,-3.693227,-411128.197118,4925261.789915
3,Centro,40.407127,-3.693677,-411178.251532,4925282.481746
4,Centro,40.407256,-3.693849,-411197.420454,4925301.309521
...,...,...,...,...,...
9466,Moncloa - Aravaca,40.469899,-3.802628,-423306.659924,4934463.574953
9467,Moncloa - Aravaca,40.469823,-3.802359,-423276.645066,4934452.379956
9468,Moncloa - Aravaca,40.469748,-3.802093,-423247.03994,4934441.47116
9469,Moncloa - Aravaca,40.469672,-3.801822,-423216.881019,4934430.303684


Save in which district is each traffic point

In [94]:
traffic_points["district"] = "None"
points = traffic_points[["utm_x", "utm_y"]]

for name in district_name:
    path = Path(df_districts[df_districts["name"] == name][["utm_x", "utm_y"]])
    points_in_path_mask = path.contains_points(points)
    traffic_points.loc[points_in_path_mask, "district"] = name

traffic_points.head()

,idelem,tipo_elem,cod_cent,nombre,st_x,st_y,latitude,longitude,utm_x,utm_y,district
0,1044,494,03FT08PM01,03FT08PM01,438963.314635,4.474734e+06,40.421001,-3.719488,-414051.481782,4.927311e+06,Moncloa - Aravaca
1,3600,494,PM30901,PM30901,443729.047369,4.473268e+06,40.408129,-3.663184,-407783.811885,4.925429e+06,Moratalaz
2,3705,494,PM41451,PM41451,439858.261097,4.471574e+06,40.392598,-3.708640,-412843.963659,4.923159e+06,Carabanchel
3,6823,494,PM41453,PM41453,439188.095183,4.470895e+06,40.386431,-3.716471,-413715.710072,4.922257e+06,Carabanchel
4,7033,495,01015,Pº Castellana S-N - Pl. Colon-Hermosilla,441569.555897,4.475502e+06,40.428107,-3.688839,-410639.639249,4.928350e+06,Salamanca


In [77]:
district_colors = sns.color_palette('Spectral', len(district_name))
np.random.shuffle(district_colors)
district_colors

[(0.7126489811610919, 0.10711264898116109, 0.28081507112648985),
 (0.3600153787004998, 0.7161860822760476, 0.6655132641291811),
 (0.5910034602076126, 0.835524798154556, 0.6442906574394464),
 (0.8141484044598232, 0.2196847366397539, 0.3048058439061899),
 (0.9963860053825452, 0.8879661668589004, 0.5610918877354863),
 (0.9817762399077278, 0.6073817762399076, 0.3457900807381776),
 (0.9557862360630527, 0.9823144944252211, 0.6800461361014996),
 (0.9330257593233372, 0.3913110342176086, 0.27197231833910035),
 (0.26405228758169935, 0.6091503267973857, 0.7098039215686275),
 (0.47427912341407163, 0.7898500576701268, 0.6459823144944252),
 (0.8202998846597465, 0.9275663206459055, 0.6126874279123413),
 (0.8758169934640523, 0.3045751633986928, 0.29411764705882354),
 (0.9982314494425221, 0.9451749327181853, 0.6570549788542868),
 (0.2941945405613224, 0.4062283737024224, 0.680968858131488),
 (0.9665513264129182, 0.49742406766628217, 0.295040369088812),
 (0.9096501345636295, 0.9638600538254518, 0.6080738177623992),
 (0.9946943483275663, 0.8092272202998846, 0.48696655132641287),
 (0.998077662437524, 0.9992310649750096, 0.7460207612456747),
 (0.7114186851211075, 0.8832756632064592, 0.6348327566320646),
 (0.21299500192233756, 0.5114186851211072, 0.730795847750865),
 (0.9928489042675894, 0.716955017301038, 0.40945790080738165)]

In [104]:
def get_color_from_palette(color):
    """ Getting colors for plotting """
    return tuple([int(c * 255) for c in color])

def get_dark_color_from_palette(color):
    """ Getting colors for plotting """
    return tuple([int(c * 200) for c in color])

In [120]:
p = figure(title="Traffic measurement points", x_axis_type="mercator", y_axis_type="mercator",
           height=700, width=800)


for name, color in zip(district_name, district_colors):
    # Districts
    source = ColumnDataSource(df_districts[df_districts["name"] == name])
    p.patch(x="utm_x", y="utm_y", color=get_color_from_palette(color), line_width=3, alpha=0.4, 
            source=source, legend_label=name, muted=True, muted_alpha=0.1)
    # Traffic points
    source = ColumnDataSource(traffic_points[traffic_points["district"] == name])
    p.circle(x="utm_x", y="utm_y", color=get_color_from_palette(color), line_width=1,
            source=source, legend_label=name, muted=True, muted_alpha=0.3, radius=30,
            line_color=get_dark_color_from_palette(color))

# Madrid Central
source = ColumnDataSource(df_districts[df_districts["name"] == "Centro"])
p.line(x="utm_x", y="utm_y", color="black", line_width=2, 
        source=source, legend_label="Madrid Central limit", muted=False, muted_alpha=0.3)

        

# Hover tooltip
TOOLTIPS = [
    ("Name", "@nombre"),
    ("District", "@district")
]
p.add_tools(HoverTool(tooltips=TOOLTIPS))

cartodb = get_provider(CARTODBPOSITRON)
p.add_tile(cartodb)
p.add_layout(p.legend[0], "right")
p.legend.click_policy = "mute"
show(p)